In [2]:
import numpy as np
import _pickle as cPickle
import os
import gzip

# First exercise: Classifying MNIST with MLPs
In this exercise you will implement a Neural Network (or MLP) and classify the MNIST digits with it.
MNIST is a "well hung" dataset that has been used a lot over the years to benchmark different classification algorithms. 
To learn more about it have a look here: http://yann.lecun.com/exdb/mnist/ .

# Data Loading
We first define a function for downloading and loading MNIST.
**WARNING**: Executing it will obviously use up some space on your machine ;). 

In [3]:
def mnist(datasets_dir='./data'):
    if not os.path.exists(datasets_dir):
        os.mkdir(datasets_dir)
    data_file = os.path.join(datasets_dir, 'mnist.pkl.gz')
    if not os.path.exists(data_file):
        print('... downloading MNIST from the web')
        try:
            import urllib
            urllib.urlretrieve('http://google.com')
        except AttributeError:
            import urllib.request as urllib
        url = 'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        urllib.urlretrieve(url, data_file)

    print('... loading data')
    # Load the dataset
    f = gzip.open(data_file, 'rb')
    try:
        train_set, valid_set, test_set = cPickle.load(f, encoding="latin1")
    except TypeError:
        train_set, valid_set, test_set = cPickle.load(f)
    f.close()

    test_x, test_y = test_set
    test_x = test_x.astype('float32')
    test_x = test_x.astype('float32').reshape(test_x.shape[0], 1, 28, 28)
    test_y = test_y.astype('int32')
    valid_x, valid_y = valid_set
    valid_x = valid_x.astype('float32')
    valid_x = valid_x.astype('float32').reshape(valid_x.shape[0], 1, 28, 28)
    valid_y = valid_y.astype('int32')
    train_x, train_y = train_set
    train_x = train_x.astype('float32').reshape(train_x.shape[0], 1, 28, 28)
    train_y = train_y.astype('int32')
    rval = [(train_x, train_y), (valid_x, valid_y), (test_x, test_y)]
    print('... done loading data')
    return rval

# Neural Network Layers
We now define "bare bone" neural network layers.
The parts marked with **TODO** are where you should finish the implementation!
Conceptually we will implement the layers as follows:

Each layer has a constructor that takes an input layer plus some additional arguments such as layer size and the activation function name. The layer then uses the provided input layer to compute the layer dimensions, weight shapes, etc. and setup all auxilliary variables.

Each layer then has to provide three functions (as defined in the Layer class below): *output_shape()*, *fprop()* and *brop()*. The output_shape function is used to figure out the shape for the next layer and the *fprop()/bprop()* functions are used to compute forward and backward passes through the network.

In [55]:
# start by defining simple helpers
def sigmoid(x):
    # sigmoid function
    return 1.0/(1.0+np.exp(-x))

def sigmoid_d(x):
    # derivative of sigmoid function
    return sigmoid(x) * (1 - sigmoid(x))

def tanh(x):
    # tangent function
    return np.tanh(x)

def tanh_d(x):
    # derivative of tangent function
    return 1 - tanh(x)**2 

def relu(x):
    # relu function
    return np.maximum(0.0, x)

def relu_d(x):
    # derivative of relu function
    return (x > 0).astype(float)

def softmax(x, axis=1):
    # to make the softmax a "safe" operation we will 
    # first subtract the maximum along the specified axis
    # so that np.exp(x) does not blow up!
    # Note that this does not change the output.
    x_max = np.max(x, axis=axis, keepdims=True)
    x_safe = x - x_max
    e_x = np.exp(x_safe)
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

def one_hot(labels):
    """this creates a one hot encoding from a flat vector:
    i.e. given y = [0,2,1]
     it creates y_one_hot = [[1,0,0], [0,0,1], [0,1,0]]
    """
    classes = np.unique(labels)
    n_classes = classes.size
    one_hot_labels = np.zeros(labels.shape + (n_classes,))
    for c in classes:
        one_hot_labels[labels == c, c] = 1
    return one_hot_labels

def unhot(one_hot_labels):
    """ Invert a one hot encoding, creating a flat vector """
    return np.argmax(one_hot_labels, axis=-1)

# then define an activation function class
class Activation(object):
    
    def __init__(self, tname):
        if tname == 'sigmoid':
            self.act = sigmoid
            self.act_d = sigmoid_d
        elif tname == 'tanh':
            self.act = tanh
            self.act_d = tanh_d
        elif tname == 'relu':
            self.act = relu
            self.act_d = relu_d
        else:
            raise ValueError('Invalid activation function.')
            
    def fprop(self, input):
        # we need to remember the last input
        # so that we can calculate the derivative with respect
        # to it later on
        self.last_input = input
        return self.act(input)
    
    def bprop(self, output_grad):
        return output_grad * self.act_d(self.last_input)

# define a base class for layers
class Layer(object):
    
    def fprop(self, input):
        """ Calculate layer output for given input 
            (forward propagation). 
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def bprop(self, output_grad):
        """ Calculate input gradient and gradient 
            with respect to weights and bias (backpropagation). 
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def output_size(self):
        """ Calculate size of this layer's output.
        input_shape[0] is the number of samples in the input.
        input_shape[1:] is the shape of the feature.
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a base class for loss outputs
# an output layer can then simply be derived
# from both Layer and Loss 
class Loss(object):

    def loss(self, output, output_net):
        """ Calculate mean loss given real output and network output. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def input_grad(self, output, output_net):
        """ Calculate input gradient real output and network output. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a base class for parameterized things        
class Parameterized(object):
    
    def params(self):
        """ Return parameters (by reference) """
        raise NotImplementedError('This is an interface class, please use a derived instance')
    
    def grad_params(self):
        """ Return accumulated gradient with respect to params. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a container for providing input to the network
class InputLayer(Layer):
    
    def __init__(self, input_shape):
        if not isinstance(input_shape, tuple):
            raise ValueError("InputLayer requires input_shape as a tuple")
        self.input_shape = input_shape

    def output_size(self):
        return self.input_shape
    
    def fprop(self, input):
        return input
    
    def bprop(self, output_grad):
        return output_grad
        
class FullyConnectedLayer(Layer, Parameterized):
    """ A standard fully connected hidden layer, as discussed in the lecture.
    """
    
    def __init__(self, input_layer, num_units, 
                 init_stddev, activation_fun=Activation('relu'), random_seed = None):
        if random_seed is not None:
            np.random.seed(random_seed)
        self.num_units = num_units
        self.activation_fun = activation_fun
        # the input shape will be of size (batch_size, num_units_prev) 
        # where num_units_prev is the number of units in the input 
        # (previous) layer
        self.input_shape = input_layer.output_size()
        # TODO ################################
        # TODO: implement weight initialization
        # TODO ################################
        # this is the weight matrix it should have shape: (num_units_prev, num_units)
        self.W = np.random.randn(self.input_shape[1], num_units) * init_stddev  #FIXME
        # and this is the bias vector of shape: (num_units)
        self.b = np.zeros((1, num_units))  #FIXME
        # create dummy variables for parameter gradients
        # no need to change these here!
        self.dW = None
        self.db = None
        self.mask = None
    
    def output_size(self):
        return (self.input_shape[0], self.num_units)
    
    def fprop(self, input):
        # TODO ################################################
        # TODO: implement forward propagation
        # NOTE: you should also handle the case were 
        #       activation_fun is None (meaning no activation)
        #       then this is simply a linear layer
        # TODO ################################################
        # you again want to cache the last_input for the bprop
        # implementation below!
        
        self.last_input = input
        a = input @ self.W + self.b
        if self.activation_fun is None:
            h = a
        else:
            h = self.activation_fun.fprop(a)
        return h
    
    
    def bprop(self, output_grad):
        """ Calculate input gradient (backpropagation). """
        # TODO ################################
        # TODO: implement backward propagation
        # TODO ###############################
        
        # HINT: you may have to divide dW and db by n
        #       to make gradient checking work 
        #       OR you divide the gradient in the output layer by n
        
        n = output_grad.shape[0]
        # accumulate gradient wrt. the parameters first
        # we will need to store these to later update
        # the network after a few forward backward passes
        # NOTE: you should also handle the case were 
        #       activation_fun is None (meaning no activation)
        # the gradient wrt. W should be stored as self.dW
        # the gradient wrt. b should be stored as self.db
        
        #raise NotImplementedError("you should implement this")
        # NOTE: self.dW is also a numpy dot product
        # if activation_fun is None, then linear activation function
        if self.activation_fun is None:
            self.dW = self.last_input.T @ np.ones_like(output_grad)/n
            self.db = np.ones_like(output_grad).sum(axis=0, keepdims = True)/n
            grad_input = output_grad @ self.W.T  
        else:
            self.dW = self.last_input.T @ self.activation_fun.bprop(output_grad)/n #FIXME
            self.db = self.activation_fun.bprop(output_grad).sum(axis = 0, keepdims = True)/n #FIXME
            # the gradient wrt. the input should be calculated here
            grad_input = self.activation_fun.bprop(output_grad) @ self.W.T
            
        return grad_input
        
    def params(self):
        return self.W, self.b

    def grad_params(self):
        return self.dW, self.db

# finally we specify the interface for output layers 
# which are layers that also have a loss function
# we will implement two output layers:
#  a Linear, and Softmax (Logistic Regression) layer
# The difference between output layers and and normal 
# layers is that they will be called to compute the gradient
# of the loss through input_grad(). bprop will never 
# be called on them!
class LinearOutput(Layer, Loss):
    """ A simple linear output layer that  
        uses a squared loss (e.g. should be used for regression)
    """
    def __init__(self, input_layer):
        self.input_size = input_layer.output_size()
        
    def output_size(self):
        return (1,)
        
    def fprop(self, input):
        return input

    def bprop(self, output_grad):
        raise NotImplementedError(
            'LinearOutput should only be used as the last layer of a Network'
            + ' bprop() should thus never be called on it!'
        )
    
    def input_grad(self, Y, Y_pred):
        # gradient of squared loss
        return (Y_pred - Y) / Y.shape[0] 
    
    def loss(self, Y, Y_pred):
        loss = 0.5 * np.square(Y_pred - Y)
        return np.mean(np.sum(loss, axis=1))

class SoftmaxOutput(Layer, Loss):
    """ A softmax output layer that calculates 
        the negative log likelihood as loss
        and should be used for classification.
    """
    
    def __init__(self, input_layer):
        self.input_size = input_layer.output_size()
        
    def output_size(self):
        return (1,)
    
    def fprop(self, input):
        return softmax(input)
    
    def bprop(self, output_grad):
        raise NotImplementedError(
            'SoftmaxOutput should only be used as the last layer of a Network'
            + ' bprop() should thus never be called on it!'
        )
    
    def input_grad(self, Y, Y_pred):
        n_samples = Y.shape[0]
        return Y_pred - Y
        

    def loss(self, Y, Y_pred):
        # Assume one-hot encoding of Y
        out = Y_pred
        # to make the loss numerically stable 
        # you should add an epsilon in the log ;)
        eps = 1e-10
        # calculate negative log likelihood
        loss = np.sum(-np.log(out + eps) * Y, axis = 1)
        loss = np.mean(loss)
        return loss
    
    

# Neural Network class
With all layers in place (and properly implemented by you) we can finally define a neural network.
For our purposes a neural network is simply a collection of layers which we will cycle through and on which we will call fprop and bprop to compute partial derivatives with respect to the input and the parameters.

Pay special attention to the *check_gradients()* function in which you should implement automatic differentiation. This function will become your best friend when checking the correctness of your implementation.

In [83]:
class NeuralNetwork:
    """ Our Neural Network container class.
        dropout: the probability of neuros being kept. The higher the less dropout
    """
    def __init__(self, layers):
        self.layers = layers
        self.dropout = 1
        self.l2_regularization = 0
        
    def _loss(self, X, Y):
        Y_pred = self.predict(X)
        loss = self.layers[-1].loss(Y, Y_pred)
        return loss

    def predict(self, X):
        """ Calculate an output Y for the given input X. """
        Y_pred = X
        for layer in self.layers:
            Y_pred = layer.fprop(Y_pred)
        return Y_pred
        
    
    def predict_train(self, X):
        """ Calculate an output Y for the given input X with dropout. """
        Y_pred = X
        # without dropout
        if self.dropout == 1:
            for layer in self.layers:
                Y_pred = layer.fprop(Y_pred)
            return Y_pred
        # with dropout
        else:
            # fully connected layers
            mask = (np.random.rand(*X.shape) < self.dropout) / self.dropout
            for layer in self.layers[1:-2]:
                Y_pred = layer.fprop(Y_pred)
                mask = (np.random.rand(*Y_pred.shape) < self.dropout) / self.dropout
                layer.mask = mask
                Y_pred = Y_pred * mask
                #layer.mask = np.random.binomial(1, self.dropout, size=Y_pred.shape)
                #Y_pred = Y_pred * layer.mask / self.dropout
            # output layer
            Y_pred = self.layers[-2].fprop(Y_pred)
            self.layers[-2].mask = np.ones(Y_pred.shape)
            Y_pred = self.layers[-1].fprop(Y_pred)
            return Y_pred 
    
    def backpropagate(self, Y, Y_pred, upto=0):
        """ Backpropagation of partial derivatives through 
            the complete network up to layer 'upto'
        """
        next_grad = self.layers[-1].input_grad(Y, Y_pred)
        m = self.layers[1].last_input.shape[0]
        if self.dropout < 1:
            # dropout
            for layer in self.layers[-2:np.maximum(0,upto-1):-1]:
                next_grad = next_grad * layer.mask
                next_grad = layer.bprop(next_grad)
                l2_regularization = self.l2_regularization/m * layer.W
                layer.dW += l2_regularization
        else:
            # without dropout
            print("enable dropout")
            for layer in self.layers[-2:np.maximum(0,upto-1):-1]:
                next_grad = layer.bprop(next_grad)
                l2_regularization = self.l2_regularization/m * layer.W
                layer.dW += l2_regularization     
        
    
    def classification_error(self, X, Y):
        """ Calculate error on the given data 
            assuming they are classes that should be predicted. 
        """
        Y_pred = unhot(self.predict(X))
        error = Y_pred != Y
        return np.mean(error)
    
    
    def sgd_epoch(self, X, Y, learning_rate, batch_size):
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        for b in range(n_batches):
            # TODO #####################################
            # Implement stochastic gradient descent here
            # TODO #####################################
            # start by extracting a batch from X and Y
            # (you can assume the inputs are already shuffled)

            # TODO: then forward and backward propagation + updates
            # HINT: layer.params() returns parameters *by reference*
            #       so you can easily update in-place
            X_batch = X[b*batch_size:(b+1)*batch_size,:]
            Y_batch = Y[b*batch_size:(b+1)*batch_size,:]
            Y_batch_pred = self.predict_train(X_batch)
            self.backpropagate(Y_batch, Y_batch_pred)
            for layer in self.layers[-2:0:-1]:
                W, b = layer.params()
                dW, db = layer.grad_params()
                layer.W = W - learning_rate * dW
                layer.b = b - learning_rate * db
        
        # deal with the rest samples
        X_rest = X[n_batches*batch_size:,:]
        Y_rest = Y[n_batches*batch_size:,:]
        Y_rest_pred = self.predict_train(X_rest)      
        self.backpropagate(Y_rest, Y_rest_pred)
        for layer in self.layers[-2:0:-1]:
            W, b = layer.params()
            dW, db = layer.grad_params()
            layer.W = W - learning_rate * dW
            layer.b = b - learning_rate * db
            
            
    def adam_epoch(self, X, Y, learning_rate, batch_size, t, parameters):
        """
        update parameters with adam
        """
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        beta1, beta2, delta = parameters
        def update_parameters(layers, learning_rate, t, parameters):
            for layer in layers[-2:0:-1]:
                W, b = layer.params()
                dW, db = layer.grad_params()
                layer.s[:-1,:] = beta1 * layer.s[:-1,:] + (1 - beta1) * layer.dW
                layer.s[[-1],:] = beta1 * layer.s[[-1],:] + (1 - beta1) * layer.db
                layer.r[:-1,:] = beta2 * layer.r[:-1,:] + (1 - beta2) * (layer.dW**2)
                layer.r[[-1],:] = beta2 * layer.r[[-1],:] + (1 - beta2) * (layer.db**2)
                s_hat = layer.s / (1 - beta1**t)
                r_hat = layer.r / (1 - beta2**t)
                delta_theta = -learning_rate * s_hat / (np.sqrt(r_hat) + delta)
                layer.W = W + delta_theta[:-1,:]
                layer.b = b + delta_theta[-1,:]

        
        for b in range(n_batches):
            X_batch = X[b*batch_size:(b+1)*batch_size,:]
            Y_batch = Y[b*batch_size:(b+1)*batch_size,:]
            Y_batch_pred = self.predict_train(X_batch)
            self.backpropagate(Y_batch, Y_batch_pred)
            t = t+1
            update_parameters(self.layers, learning_rate, t, parameters)
        
        # deal with the rest samples
        X_rest = X[n_batches*batch_size:,:]
        Y_rest = Y[n_batches*batch_size:,:]
        Y_rest_pred = self.predict_train(X_rest)     
        self.backpropagate(Y_rest, Y_rest_pred)
        t = t+1
        update_parameters(self.layers, learning_rate, t, parameters)      
    
    def gd_epoch(self, X, Y):
        # TODO ##################################################
        # Implement batch gradient descent here
        # A few hints:
        #   There are two strategies you can follow:
        #   Either shove the whole dataset throught the network
        #   at once (which can be problematic for large datasets)
        #   or run through it batch wise as in the sgd approach
        #   and accumulate the gradients for all parameters as
        #   you go through the data. Either way you should then
        #   do one gradient step after you went through the
        #   complete dataset!
        # TODO ##################################################
        Y_pred = self.predict_train(X)
        self.backpropagate(Y, Y_pred)
        for layer in self.layers[-2:0:-1]:
            W, b = layer.params()
            dW, db = layer.grad_params()
            W = W - learning_rate * dW
            b = b - learning_rate * db
        
    
    def train(self, X, Y, X_valid, Y_valid, learning_rate=0.1, max_epochs=100, batch_size=64,
              descent_type="sgd", dropout=1, y_one_hot=True, l2_regularization=0):

        """ Train network on the given data. """
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        self.train_loss = []
        self.valid_loss = []
        self.train_error = []
        self.valid_error = []
        """        
        if (X_test is not None) and (Y_test is not None):
            self.test_error = []"""
        # check and set the l2 regularization
        if l2_regularization < 0:
            raise RegularizationError("Regularization parameter should be greater or equal to 0")
        else:
            self.l2_regularization = l2_regularization
        # check and set the drop out parameters
        if dropout <= 0 or dropout > 1:
            raise DropoutError("Dropout parameter shoud between 0 and 1")
        else:
            self.dropout = dropout
        t = 0 # set the parameter for adam
        if y_one_hot:
            Y_train = one_hot(Y)
            Y_valid_hot = one_hot(Y_valid)
        else:
            Y_train = Y
        
        print("... starting training")
        # initialize the parameters we need for adam
        if descent_type == "adam":
            for layer in self.layers[-2:0:-1]:
                layer.s = np.zeros((layer.W.shape[0]+1, layer.W.shape[1]))
                layer.r = np.zeros((layer.W.shape[0]+1, layer.W.shape[1]))
            beta1 = 0.9
            beta2 = 0.999
            delta = 1e-8
            parameters = [beta1, beta2, delta]
        
        for e in range(max_epochs+1):
            # trainning
            if descent_type == "sgd":
                self.sgd_epoch(X, Y_train, learning_rate, batch_size)
            elif descent_type == "adam":
                self.adam_epoch(X, Y_train, learning_rate, batch_size, t, parameters)
            elif descent_type == "gd":
                self.gd_epoch(X, Y_train, learning_rate)
            else:
                raise NotImplementedError("Unknown gradient descent type {}".format(descent_type))
            
            # Output error on the training data
            train_loss = self._loss(X, Y_train)
            train_error = self.classification_error(X, Y)
            print('epoch {:.4f}, loss {:.4f}, train error {:.4f}'.format(e, train_loss, train_error))
            self.train_loss.append(train_loss)
            self.train_error.append(train_error)
            # TODO ##################################################
            # compute error on validation data:
            # simply make the function take validation data as input
            # and then compute errors here and print them
            # TODO ##################################################
            validation_loss = self._loss(X_valid, Y_valid_hot)
            validation_error = self.classification_error(X_valid, Y_valid)
            print('epoch {:.4f}, validation loss {:.4f}, validation error {:.4f}'.format(e, validation_loss, validation_error))            
            self.valid_loss.append(validation_loss)
            self.valid_error.append(validation_error)
            """            
            if X_test is not None:
                test_error = self.classification_error(X_test, Y_test)
                self.test_error.append(test_error)
            """
            
    def check_gradients(self, X, Y):
        """ Helper function to test the parameter gradients for
        correctness. """
        for l, layer in enumerate(self.layers):
            if isinstance(layer, Parameterized):
                print('checking gradient for layer {}'.format(l))
                for p, param in enumerate(layer.params()):
                    # we iterate through all parameters
                    param_shape = param.shape
                    # define functions for conveniently swapping
                    # out parameters of this specific layer and 
                    # computing loss and gradient with these 
                    # changed parametrs
                    def output_given_params(param_new):
                        """ A function that will compute the output 
                            of the network given a set of parameters
                        """
                        # copy provided parameters
                        param[:] = np.reshape(param_new, param_shape)
                        # return computed loss
                        return self._loss(X, Y)

                    def grad_given_params(param_new):
                        """A function that will compute the gradient 
                           of the network given a set of parameters
                        """
                        # copy provided parameters
                        param[:] = np.reshape(param_new, param_shape)
                        # Forward propagation through the net
                        Y_pred = self.predict(X)
                        # Backpropagation of partial derivatives
                        self.backpropagate(Y, Y_pred, upto=1)
                        # return the computed gradient 
                        return np.ravel(self.layers[l].grad_params()[p])

                    # let the initial parameters be the ones that
                    # are currently placed in the network and flatten them
                    # to a vector for convenient comparisons, printing etc.
                    param_init = np.ravel(np.copy(param))
                    # To debug you network's gradients use scipys
                    # gradient checking!
                    epsilon = 1e-5
                    import scipy.optimize
                    err = scipy.optimize.check_grad(output_given_params, 
                          grad_given_params, param_init) 
                    print('diff scipy {:.2e}'.format(err))
                    assert(err < epsilon)
                    
                    # reset the parameters to their initial values
                    param[:] = np.reshape(param_init, param_shape)

# Gradient Checking
After implementing everything it is always a good idea to setup some layers and perform gradient
checking on random data. **Note** that this is only an example! It is not a useful network architecture ;). We also expect you to play around with this to test all your implemented components.

In [39]:
input_shape = (5, 10)
n_labels = 6
layers = [InputLayer(input_shape)]

layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=15,
                init_stddev=0.1,
                activation_fun=Activation('relu'),
                random_seed = 123
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=6,
                init_stddev=0.1,
                activation_fun=Activation('tanh'),
                random_seed = 321
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=n_labels,
                init_stddev=0.1,
                activation_fun=Activation('relu'),
                random_seed = 132
))
layers.append(SoftmaxOutput(layers[-1]))
nn = NeuralNetwork(layers)

In [40]:
# create random data
X = np.random.normal(size=input_shape)
# and random labels
Y = np.zeros((input_shape[0], n_labels))
for i in range(Y.shape[0]):
    idx = np.random.randint(n_labels)
    Y[i, idx] = 1.

In [41]:
nn.check_gradients(X, Y)

checking gradient for layer 1
diff scipy 1.48e-07
diff scipy 5.53e-08
checking gradient for layer 2
diff scipy 1.18e-07
diff scipy 2.84e-08
checking gradient for layer 3
diff scipy 9.67e-08
diff scipy 3.58e-08


# Training on MNIST
Finally we can let our network run on the MNIST dataset!

First load the data and reshape it.

In [27]:
# load
Dtrain, Dval, Dtest = mnist()
X_train, y_train = Dtrain
X_valid, y_valid = Dval
X_test, y_test = Dtest

... loading data
... done loading data


*Dtrain* contains 50k images which are of size 28 x 28 pixels. Hence:

In [28]:
print("X_train shape: {}".format(np.shape(X_train)))
print("y_train shape: {}".format(np.shape(y_train)))

X_train shape: (50000, 1, 28, 28)
y_train shape: (50000,)


y_train will automatically be converted in the *train()* function to one_hot encoding.


But we need to reshape X_train, as our Network expects flat vectors of size 28*28 as input!

In [29]:
X_train = X_train.reshape(X_train.shape[0], -1)
print("Reshaped X_train size: {}".format(X_train.shape))
X_valid = X_valid.reshape((X_valid.shape[0], -1))
print("Reshaped X_valid size: {}".format(X_valid.shape))
print("y_valid shape: {}".format(np.shape(y_valid)))
X_test = X_test.reshape((X_test.shape[0], -1))
print("Reshaped X_test size: {}".format(X_test.shape))
print("y_test shape: {}".format(np.shape(y_test)))

Reshaped X_train size: (50000, 784)
Reshaped X_valid size: (10000, 784)
y_valid shape: (10000,)
Reshaped X_test size: (10000, 784)
y_test shape: (10000,)


Ah, much better ;-)! 

Now we can finally really start training a Network!


I pre-defined a small Network for you below. Again This is not really a good default and will not produce state of the art results. Please play around with this a bit. See how different activation functions and training procedures (gd / sgd) affect the result.

In [91]:
#runcode
import time
#np.seterr(all='raise')
# Setup a small MLP / Neural Network
# we can set the first shape to None here to indicate that
# we will input a variable number inputs to the network
input_shape = (None, 28*28)
layers = [InputLayer(input_shape)]
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=64,
                init_stddev=0.01,
                activation_fun=Activation('relu'),
                random_seed = 0
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=32,
                init_stddev=0.01,
                activation_fun=Activation('relu'),
                random_seed = 1
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=32,
                init_stddev=0.01,
                activation_fun=Activation('relu'),
                random_seed = 78
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=16,
                init_stddev=0.01,
                activation_fun=Activation('relu'),
                random_seed = 5123
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=10,
                init_stddev=0.01,
                # last layer has no nonlinearity 
                # (softmax will be applied in the output layer)
                activation_fun=None,
                random_seed = 6
))
layers.append(SoftmaxOutput(layers[-1]))

nn = NeuralNetwork(layers)
# Train neural network
t0 = time.time()
nn.train(X_train, y_train, X_valid, y_valid, learning_rate=0.01, 
         max_epochs=40, batch_size=64, y_one_hot=True, descent_type = "adam", dropout = 0.75)
t1 = time.time()
print('Duration: {:.1f}s'.format(t1-t0))

... starting training
epoch 0.0000, loss 0.9447, train error 0.3336
epoch 0.0000, validation loss 0.9303, validation error 0.3300
epoch 1.0000, loss 0.8894, train error 0.3054
epoch 1.0000, validation loss 0.8861, validation error 0.3022
epoch 2.0000, loss 0.7927, train error 0.2705
epoch 2.0000, validation loss 0.7894, validation error 0.2685
epoch 3.0000, loss 0.7063, train error 0.2579
epoch 3.0000, validation loss 0.7099, validation error 0.2577
epoch 4.0000, loss 0.6502, train error 0.1723
epoch 4.0000, validation loss 0.6606, validation error 0.1737
epoch 5.0000, loss 0.6417, train error 0.1737
epoch 5.0000, validation loss 0.6491, validation error 0.1776
epoch 6.0000, loss 0.6125, train error 0.1703
epoch 6.0000, validation loss 0.6208, validation error 0.1714
epoch 7.0000, loss 0.6112, train error 0.1717
epoch 7.0000, validation loss 0.6246, validation error 0.1752
epoch 8.0000, loss 0.5983, train error 0.1653
epoch 8.0000, validation loss 0.6246, validation error 0.1668
epoch 

KeyboardInterrupt: 

In [85]:
no_train_loss = nn.train_loss
no_train_error = nn.train_error
no_valid_loss = nn.valid_loss
no_valid_loss = nn.valid_error
no_test_error = nn.classification_error(X_test, y_test)
print(no_test_error)

0.029


In [75]:
with_train_loss = nn.train_loss
with_train_error = nn.train_error
with_valid_loss = nn.valid_loss
with_valid_loss = nn.valid_error
with_test_error = nn.classification_error(X_test, y_test)
print(with_test_error)

0.0453


# Figure out a reasonable Network that achieves good performance
As the last part of this task, setup a network that works well and gets reasonable accuracy, say ~ 1-3 percent error on the **validation set**. 
Train this network on the complete data and compute the **test error**. 

Visualize the validation loss and training loss for each iteration in a plot, e.g. using matplotlib

In [ ]:
a = np.array([[1,2],[3,4]])

In [ ]:
print(a[:-1,:])

In [ ]:
a = [1,2,3]

In [ ]:
print(a[0:-1])

In [ ]:
    def predict_train(self, X):
        """ Calculate an output Y for the given input X with dropout. """
        Y_pred = X
        # without dropout
        if self.dropout == 1:
            for layer in self.layers:
                Y_pred = layer.fprop(Y_pred)
            return Y_pred
        # with dropout
        else:
            p = self.dropout
            """            
            # input layer
            Y_pred = self.layers[0].fprop(Y_pred)
            """
            # fully connected layers
            #mask = (np.ones(*Y_pred.shape) < p) / p
            mask = (np.random.rand(*Y_pred.shape) < p) /p
            for layer in self.layers[1:-1]:
                layer.mask = mask
                Y_pred = layer.fprop(Y_pred)
                mask = (np.random.rand(*Y_pred.shape) < p) / p
                Y_pred = Y_pred * mask
            # output layer
            self.layers[-1].mask = mask
            Y_pred = self.layers[-1].fprop(Y_pred)
            return Y_pred
        

In [ ]:
for i, k in enumerate(a[1:]):
    print(i)
    print(k)

In [ ]:
       """
       else:
            # add mask
            next_grad = self.layers[-1].input_grad(Y, Y_pred)# * self.layers[-1].mask# + l2_regularization_cost
            for layer in self.layers[-2:np.maximum(0,upto-1):-1]:
                next_grad = layer.bprop(next_grad)"""

$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} \tag{1}$$ To: $$J_{regularized} = \small \underbrace{-\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} }_\text{cross-entropy cost} + \underbrace{\frac{1}{m} \frac{\lambda}{2} \sum\limits_l\sum\limits_k\sum\limits_j W_{k,j}^{[l]2} }_\text{L2 regularization cost} \tag{2}$$